# Imports

In [1]:
# !pip install seaborn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import tensorflow as tf
import polars as pl
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder, StandardScaler, normalize
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, auc
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import keras
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
from time import time
import numpy as np
from sklearn.svm import LinearSVC

2024-04-15 07:08:49.541855: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-15 07:08:49.542056: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-15 07:08:49.707296: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Dataset

In [ ]:
data = pd.read_csv("/kaggle/input/cleaned-malware-detection/Cleaned_malware.csv", low_memory=False)
data.head()

In [ ]:
data.drop("Unnamed: 0", axis = 1, inplace = True)

In [ ]:
data.info()

In [ ]:
data.describe()

# Data Cleaning

In [ ]:
data.isna()

In [ ]:
# data["Census_IsAlwaysOnAlwaysConnectedCapable"].isnull().sum() / len(data)

In [ ]:
# data.columns[0]

In [ ]:
# for c in data.columns:
#     if ((data[c].isnull().sum() * 100) / len(data)) > 70:
#         data.drop(c, inplace=True, axis = 1)

In [ ]:
data.head()

In [ ]:
# data.isnull().sum(axis = 1)

In [ ]:
# data = data[data.isnull().sum(axis = 1) == 0]
# data.head()

In [ ]:
# print(data.shape)
# max(data.isnull().sum())

In [ ]:
# data.to_csv("Cleaned_malware.csv")

In [ ]:
# data[data['Firewall'] == 1.0]['HasDetections'].value_counts()

# EDA

In [ ]:
sns.histplot(data['ProductName'])

In [ ]:
# How many computers had firewall enabled?
sns.countplot(x = data[data['Firewall'] == 1.0]['HasDetections'])

In [ ]:
# The version of OS used
(data['Census_OSVersion'].value_counts() > 10000).head()

In [ ]:
data[data['HasDetections'] == 1]['Census_TotalPhysicalRAM'].value_counts().head(10)

In [ ]:
# How much RAM did infected PCs have?
plt.figure(figsize=(20, 8))
plt.tight_layout()
x = dict(data[data['HasDetections'] == 1]['Census_TotalPhysicalRAM'].value_counts().head(10)).keys()
y = dict(data[data['HasDetections'] == 1]['Census_TotalPhysicalRAM'].value_counts().head(10)).values()
sns.barplot(x = list(x), y = list(y))

In [ ]:
data[data['HasDetections'] == 1]['HasTpm'].value_counts()

In [ ]:
# How many infected PCs had TPM?
plt.tight_layout()
x = dict(data[data['HasDetections'] == 1]['HasTpm'].value_counts()).keys()
y = dict(data[data['HasDetections'] == 1]['HasTpm'].value_counts()).values()
sns.barplot(x = list(x), y = list(y))

In [ ]:
data['Wdft_IsGamer'].value_counts()

In [ ]:
# How many infected PCs had gamer device configuration?
plt.tight_layout()
x = dict(data[data['HasDetections'] == 1]['Wdft_IsGamer'].value_counts()).keys()
y = dict(data[data['HasDetections'] == 1]['Wdft_IsGamer'].value_counts()).values()
sns.barplot(x = list(x), y = list(y))

In [ ]:
# How many infected computers had any antivirus protection?
data[data['HasDetections'] == 1]['IsProtected'].value_counts()

In [ ]:
# Status of Microsoft Smartscreen on the infected computers
data[data['HasDetections'] == 1]['SmartScreen'].value_counts()

# **Data Preprocessing**

### Label Encoding

In [ ]:
# data.reset_index(drop = True, inplace = True)
# data.head()

In [ ]:
# data['AppVersion'].dtypes == 'object'
data.dtypes.value_counts()

In [ ]:
len(data['HasDetections'].value_counts())

In [ ]:
le = LabelEncoder()
numericCols = []
for c in data.columns:
    if data[c].dtypes != 'object' and len(data[c].value_counts()) > 2:
        numericCols.append(c)
    if data[c].dtypes == 'object':
        data[c] = le.fit_transform(data[c])

print(data.dtypes.value_counts())
print(numericCols)

In [ ]:
# for c in data.columns:
#     if data[c].dtypes == 'float64':
#         print(data[c])

### Standard Scaling

In [ ]:
# df_copy = data.copy()
# df_copy = df_copy.drop(columns=['HasDetections'])
scaler = StandardScaler()
# data_scaled = pd.DataFrame(scaler.fit_transform(df_copy), columns=df_copy.columns)
# data_scaled['HasDetections'] = data['HasDetections']

In [ ]:
for c in numericCols:
    data[c] = scaler.fit_transform(data[c].values.reshape(-1, 1))

In [ ]:
data.head()

### Normalisation

In [ ]:
list(data.columns)[:-1]

In [ ]:
# data_norm = pd.DataFrame(normalize(data.drop("HasDetections", axis = 1)), columns=list(data.columns)[:-1])
# data_norm["HasDetections"] = data["HasDetections"]
# print(data_norm.shape)
# data_norm.head()

In [ ]:
# data_norm["HasDetections"].isnull().sum()

# Feature Engineering

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA

def pca_top_features(dataframe, target_column, n_components=50):
    # Separate features and target column
    X = dataframe.drop(columns=[target_column])
    y = dataframe[target_column]
    
    # Perform PCA
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X)
    
    # Get indices of top features
    top_feature_indices = pca.components_.argsort(axis=1)[:, ::-1][:, :n_components]
    
    # Extract top features
    top_features = X.columns[top_feature_indices.flatten()].unique()
    
    return top_features

# Example usage
# Assuming 'df' is your pandas dataframe and 'target_column' is the column you want to predict
top_features = pca_top_features(data, 'HasDetections')
print("Top 50 features affecting the target column:", top_features)


# Splitting the dataset

In [38]:
x = data.drop("HasDetections", axis = 1)
y = data['HasDetections']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)

# Models

#### 1. Linear SVC

In [ ]:
modelSVC = LinearSVC()
start = time()
modelSVC.fit(x_train, y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_SVC = modelSVC.predict(x_test)
acc = accuracy_score(pred_SVC , y_test)
cm = confusion_matrix(pred_SVC , y_test)
cr = classification_report(pred_SVC , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

#### 2. LGBM

In [ ]:
modelLGBM = LGBMClassifier()
start = time()
modelLGBM.fit(x_train , y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_LGBM = modelLGBM.predict(x_test)
acc = accuracy_score(pred_LGBM , y_test)
cm = confusion_matrix(pred_LGBM , y_test)
cr = classification_report(pred_LGBM , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

#### 3. Random Forest

In [ ]:
model_RF = RandomForestClassifier(n_jobs=-1)
start = time()
model_RF.fit(x_train, y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_RF = model_RF.predict(x_test)
acc = accuracy_score(pred_RF , y_test)
cm = confusion_matrix(pred_RF , y_test)
cr = classification_report(pred_RF , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

#### 4. ANN

In [ ]:
# model = keras.Sequential([
#     keras.layers.Dense(256, activation='relu', input_shape = (304,)),
#     keras.layers.Dense(512, activation='relu'),
#     keras.layers.Dropout(0.5),
#     keras.layers.Dense(1, activation='sigmoid')
# ])

# model.compile(optimizer=keras.optimizers.Adam(), loss=tf.keras.losses.BinaryCrossentropy(), metrics=keras.metrics.CategoricalAccuracy())

# start = time()
# history = model.fit(x_train, y_train, epochs=5, batch_size=32, use_multiprocessing=True, validation_split=0.5)
# end = time()
# print("--------------------------------------------------------")
# print(f"Time taken: {end - start} seconds")
# print("--------------------------------------------------------")
# pred_ANN = model.predict(x_test)

#### 5. Logistic Regression

In [ ]:
model_lr = LogisticRegression(n_jobs = -1, max_iter=1000)
start = time()
model_lr.fit(x_train, y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_lr = model_lr.predict(x_test)
acc = accuracy_score(pred_lr , y_test)
cm = confusion_matrix(pred_lr , y_test)
cr = classification_report(pred_lr , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

#### 6. Decision Trees

In [ ]:
model_DT = DecisionTreeClassifier()
start = time()
model_DT.fit(x_train, y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_DT = model_DT.predict(x_test)
acc = accuracy_score(pred_DT , y_test)
cm = confusion_matrix(pred_DT , y_test)
cr = classification_report(pred_DT , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

#### 7. K-Nearest Neighbours

In [ ]:
model_KN = KNeighborsClassifier()
start = time()
model_KN.fit(x_train, y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_KN = model_KN.predict(x_test)
acc = accuracy_score(pred_KN , y_test)
cm = confusion_matrix(pred_KN , y_test)
cr = classification_report(pred_KN , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

#### 8. MLP Classifier

In [ ]:
model_NN = MLPClassifier()
start = time()
model_NN.fit(x_train, y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_NN = model_NN.predict(x_test)
acc = accuracy_score(pred_NN , y_test)
cm = confusion_matrix(pred_NN , y_test)
cr = classification_report(pred_NN , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

#### 9. XGBoost

In [ ]:
model_XG = xgb.XGBClassifier()
start = time()
model_XG.fit(x_train,y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_XG = model_XG.predict(x_test)
acc = accuracy_score(pred_XG , y_test)
cm = confusion_matrix(pred_XG , y_test)
cr = classification_report(pred_XG , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

#### 10. AdaBoost

In [ ]:
model_ada = AdaBoostClassifier()
start = time()
model_ada.fit(x_train,y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_ada = model_ada.predict(x_test)
acc = accuracy_score(pred_ada , y_test)
cm = confusion_matrix(pred_ada , y_test)
cr = classification_report(pred_ada , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

#### 12. Gaussian Naive Bayes

In [ ]:
model_gnb = GaussianNB()
start = time()
model_gnb.fit(x_train,y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_gnb = model_gnb.predict(x_test)
acc = accuracy_score(pred_gnb , y_test)
cm = confusion_matrix(pred_gnb , y_test)
cr = classification_report(pred_gnb , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")